### Random Number Generator (Weighted by arbitrary probabilities)
Implement the method `nextNum()` and a minimal but effective set of unit tests. Implement in the language of your choice, Python is preferred, but Java and other languages are completely fine. Make sure your code is exemplary, as if it was going to be shipped as part of a production system.
As a quick check, given Random Numbers are `[-1, 0, 1, 2, 3]` and Probabilities are `[0.01, 0.3, 0.58, 0.1, 0.01]` if we call `nextNum()` 100 times we may get the following results.

|n |frequency|
|-:|--------:|
|-1| 1 times |
|0 |22 times |
|1 |57 times |
|2 |20 times |
|3 |0 times  |

As the results are random, these particular results are unlikely.

Languages Python
You may use `random.random()` which returns a pseudo random number between 0 and 1.

In [1]:
import random
import numpy as np
from bisect import bisect_right
from typing import Iterable
from math import fsum

In [4]:
class RandomGen(object):
    
    def guard(self, candidates, probabilities):
        """
        Guard statements to check that inputs are valid
        
        candidates: Iterable
        probabilities: Iterable[float]        
        """
        # candidates and probabilities must be same dimension
        if len(candidates) != len(probabilities):
            raise ValueError("Dimensions of candidates and probabilities are different.")
        
        # candidates and probabilities must not be empty
        if len(candidates) == 0 or len(probabilities) == 0:
            raise ValueError(f"Candidates and probabilities must not be empty")
        
        # probabilities must be positive
        negatives = len(list(filter(lambda x: x < 0, probabilities)))
        if negatives > 0:
            raise ValueError(f"Probabilities contains {negatives} negative number(s)")
        
        # probabilities must all add up to 1
        total_prob = fsum(probabilities)
        if total_prob != 1.0:
            raise ValueError(f"Probabilities must all add up to 1 instead of {total_prob}")

    def __init__(self, candidates: Iterable, probabilities: Iterable[float]): 
        self.guard(candidates, probabilities)
        self._cumulative_totals = np.cumsum(probabilities)
        self._random_nums = candidates
        self._probabilities = probabilities
    
    def next_num(self):
        """
        Returns one of the randomNums. When this method is called
        multiple times over a long period, it should return the
        numbers roughly with the initialized probabilities.
        """
        # use binary search to insert random into a sorted list
        rnd = random.random() * self._cumulative_totals[-1]
        i = bisect_right(self._cumulative_totals, rnd)
        return self._random_nums[i]
    
    def __call__(self):
        return self.next_num()

In [11]:
from unittest import *
from numpy.testing import assert_almost_equal

class TestCases(TestCase):
        
    def test_empty(self):
        self.assertRaises(ValueError, RandomGen, [], [])
        self.assertRaises(ValueError, RandomGen, ['one'], [])
        self.assertRaises(ValueError, RandomGen, [], [1])
    
    def test_unequal_dimensions(self):
        self.assertRaises(ValueError, RandomGen, ['one', 'two'], [1])
        self.assertRaises(ValueError, RandomGen, ['one'], [0.5,0.5])
        with self.assertRaises(ValueError) as e: 
            RandomGen(['one'], [0.5,0.5])
        self.assertEqual(str(e.exception), 'Dimensions of candidates and probabilities are different.')
    
    def test_invalid_probabilities(self):
        self.assertRaises(ValueError, RandomGen, ['one'], [-1])
        self.assertRaises(ValueError, RandomGen, ['one'], [1.1])             
        self.assertRaises(ValueError, RandomGen, ['one', 'two'], [-0.1,1.1])
        
    def test_invalid_total_probability(self):
        self.assertRaises(ValueError, RandomGen, ['one', 'two'], [0.1,0.2])
        self.assertRaises(ValueError, RandomGen, ['one', 'two'], [0.9,0.2])
        
    def test_arguments_iterable(self):
        self.assertRaises(TypeError, RandomGen, 1, 1)
        self.assertRaises(TypeError, RandomGen, [1], 1)
        self.assertRaises(TypeError, RandomGen, 1, [1])
    
    def test_probabilities_iterable_numerics(self):
        self.assertRaises(TypeError, RandomGen, 1, 'A')
        self.assertRaises(TypeError, RandomGen, [1], ['A'])
        self.assertRaises(TypeError, RandomGen, [1,2], [1,'A'])
        
    def test_fsum_precision_on_probabilities(self):
        weighted_random = RandomGen([-1, 0, 1, 2, 3], [0.01, 0.3, 0.58, 0.1, 0.01])
        
    def test_distribution(self):
        candidates = [-1, 0, 1, 2, 3]
        given_probs = [0.01, 0.3, 0.58, 0.1, 0.01]
        sample_size = 10000
        
        weighted_random = RandomGen(candidates, given_probs)
        samples = [weighted_random() for _ in range(sample_size)]
        unique, counts = np.unique(samples, return_counts=True)
        totals = sum(counts)
        actual_probs = counts/totals
        
        assert_almost_equal(actual_probs, given_probs, decimal=2)
        
        
test_cases = TestCases()
test_suite = TestLoader().loadTestsFromModule(test_cases)
TextTestRunner().run(test_suite)

........
----------------------------------------------------------------------
Ran 8 tests in 0.032s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

#### Q. Please describe how you might implement this more "pythonically"
A: By implementing iterator and generator functions `__iter__` and `__next__` and using the `yield` keyword so that the weighted random function appears like a sequence.